# 다양한 모델을 결합한 앙상블
- 앙상블
- 다수결 투표 앙상블(Voting)
- 배깅(Bagging)
- 그래이언트 부스팅 & XGBoost
- 모델성능 평가 및 비교

단일모델의 한계
- 과대적합, 과소적합
- 높은 분산
- 높은 편향

---

## 앙상블 (집단지성)
- 배경: 같은 알고리즘, 다른 데이터셋(Bootstrap)  `RandomForest` 분산감소
- 부스팅: 순차적으로 약한 학습기를 강화           `AdaBoost`, `XGBoost` 편향감소
- 스태깅: 다른 알고리즘을 메타 모델로 학습        `Stacked Generalization` 일반화 성능

### 1. Voting
- `Hard Voting`: 다수결 투표
- `Soft Voting`: 가중치 투표 - 확률을 보고 결정함. 보통 Hard Voting보다 성능이 좋음
### 2. Bagging
- `Bootstrap Aggregation`
- `Bootstrap`: 원본 데이터 1000개 중 중복을 허용해 1000번을 뽑아, 훈련세트 만들기
- `Aggregation`: 순차적으로 학습하고난 후 여러개의 모델들을 투표해 최종결론 내리기
- 대표모델: `RandomForest`
- 효과: 과적합 방지 = 모델의 분산을 줄임
### 3. Boosting
**앞선 모델의 오답을 보완하면서 순차적으로 학습하는 방식**
> 1. 모델 1 (기본 학습)
- 전체 데이터를 학습하고 예측 수행  
- 오답(틀린 샘플)을 식별  
>  2. 가중치 조정
- 모델 1이 틀린 데이터에 **더 높은 가중치(중요도)** 부여  
> 3. 모델 2 (보완 학습)
- 가중치가 높은, 즉 **틀리기 쉬운 데이터**에 더 집중하여 학습  
> 4. 모델 3 이후 (반복 보완)
- 이전 모델(모델 1, 2)이 공통으로 틀린 문제에 집중  
- 오답에 다시 가중치 부여 → 반복적으로 학습 성능 개선  
> 5. 최종 단계 (결합)
- 각 모델의 예측 결과를 **가중 평균(Weighted Average)** 으로 결합  
- **성능이 좋은 모델일수록 가중치↑**  
- 이를 통해 **편향(Bias)을 줄이고 예측 성능 향상**

### 4. Stacking
- 1 base model: 여러개의 모델 훈련
- 2 meta model:
    - 1단계 모델들이 예측한 값들을 모음
    - 예측한 값을 훈련데이터(특성)로 사용
- 결론:
    - 새로운 데이터를 최종모델(Meta-model)(ex.로지스틱회귀) 도출

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
import numpy as np


# 데이터 생성
X,y = make_classification(n_samples=500, n_features=2, random_state=42, n_redundant=0)
np.unique(y,return_counts=True)

(array([0, 1]), array([249, 251]))

In [ ]:
# 훈련/테스트 데이터 나누기
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)


# 개별모델 생성
lr = LogisticRegression(random_state=42)
svm = SVC(random_state=42, probability=True)
knn = KNeighborsClassifier()

lr.fit(X_train,y_train)
svm.fit(X_train,y_train)
knn.fit(X_train,y_train)



# Voting 앙상블
voting_hard = VotingClassifier(estimators=[('lr',lr),('svm',svm),('knn',knn)], voting='hard')
voting_soft = VotingClassifier(estimators=[('lr',lr),('svm',svm),('knn',knn)], voting='soft')

voting_hard.fit(X_train,y_train)
voting_soft.fit(X_train,y_train)

models = [
    ('lr',lr),
    ('svm',svm),
    ('knn',knn),
    ('voting_hard',voting_hard),
    ('voting_soft',voting_soft)
]


for label, model in models:
    print(model.score(X_test,y_test))

0.88
0.89
0.93
0.9


AttributeError: This 'SVC' has no attribute 'predict_proba'